# TensorFlow Serving with Prometheus Metrics

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80`

## Install Seldon Analytics

In [ ]:
!helm install seldon-core-analytics ../../../helm-charts/seldon-core-analytics -n seldon-system --wait

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
import json
import time

## Tensorflow Serving Prometheus Metrics

Create a configmap for Tensorflow Serving monitoring configuration

In [ ]:
!pygmentize tfconfig.yaml

In [ ]:
!kubectl create -f tfconfig.yaml

In [ ]:
!pygmentize tfserving_metrics.yaml

Note:

  * A port with name starting with "metrics-" is needed to allow scraping by Seldon's default analytics stack.
  * The ConfigMap is mounted as a volume and used in the command line argument with `--monitoring_config`

In [ ]:
!kubectl create -f tfserving_metrics.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=rest-tfserving \
                                 -o jsonpath='{.items[0].metadata.name}')

In [ ]:
X=!curl -s -d '{"instances": [1.0, 2.0, 5.0]}' \
   -X POST http://localhost:8003/seldon/seldon/rest-tfserving/v1/models/halfplustwo/:predict \
   -H "Content-Type: application/json"
d=json.loads("".join(X))
print(d)
assert(d["predictions"][0] == 2.5)

In [ ]:
print("Waiting so metrics can be scraped")
time.sleep(3)

In [ ]:
responseRaw=!kubectl run --quiet=true -it --rm curl --image=radial/busyboxplus:curl --restart=Never -- curl -s seldon-core-analytics-prometheus-seldon.seldon-system/api/v1/query?query=:tensorflow:core:graph_runs

In [ ]:
response = json.loads(responseRaw[0])
print(response)
assert (
    response["data"]["result"][0]["metric"]["__name__"] == ":tensorflow:core:graph_runs"
)

In [ ]:
!kubectl delete -f tfserving_metrics.yaml

In [ ]:
!kubectl delete -f tfconfig.yaml